In [101]:
! pip install pandas
! pip install numpy
! pip install statsmodels

You should consider upgrading via the '/Users/sennakim/envcatie/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/sennakim/envcatie/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/sennakim/envcatie/bin/python -m pip install --upgrade pip' command.


In [102]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm

In [103]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'sklearn'

In [104]:

def process_file(file_path, text_columns_start_row=0):
    """
    Processes a text file to handle numeric columns and filter NaN values.
    
    Parameters:
    - file_path: Path to the text file.
    - text_columns_start_row: Row index from where to start processing text columns (default is 1).
    
    Returns:
    - recombined_df: A DataFrame with filtered numeric and text columns.
    """
    # Step 1: Load the data into a pandas DataFrame
    df = pd.read_csv(file_path, delimiter='\t', low_memory=False)
    
    # Step 2: Select the first two text columns
    df_first_two_columns = df.iloc[text_columns_start_row:, :2]  # First two columns
    
    # Step 3: Select the numeric columns (all except the first two and last two)
    df_numeric = df.iloc[text_columns_start_row:, 2:-2]  # Select numeric columns dynamically
    
    # Step 4: Apply to_numeric to convert non-numeric values to NaN. Exclude the description row and then add it back.
    first_row = df_numeric.iloc[0]
    rest_of_df = df_numeric.iloc[1:]
    df_numeric = rest_of_df.apply(pd.to_numeric, errors='coerce')
    df_numeric = pd.concat([pd.DataFrame([first_row]), df_numeric], ignore_index=True)

    # Step 5: Retain the last two text columns
    df_last_two_columns = df.iloc[text_columns_start_row:, -2:]  # Last two columns
    
    # Step 6: Filter rows with no NaN values in the numeric columns
    non_nan_filtered = df_numeric.dropna()
    
    # Step 7: Filter out columns that have all NaN values in the numeric subset

    # Function to check if all elements except the first one are NaN
    def should_drop(column):
        return column[1:].isna().all()

    # Identify columns to drop
    columns_to_drop = [col for col in df_numeric.columns if should_drop(df_numeric[col])]

    # Drop those columns
    non_nan_columns = df_numeric.drop(columns=columns_to_drop)
    
    # Step 8: Recombine the numeric subset, first two text columns, and last two text columns
    recombined_df = pd.concat([df_first_two_columns, non_nan_columns, df_last_two_columns], axis=1)

    # Step 9: Check that collection_title or promoted_subjectkey are both there, otherwise print a warning
    description_row = recombined_df.iloc[0]
    desc_contains_collection_title = description_row.apply(lambda x: 'collection_title' in str(x)).any()
    desc_contains_promoted_subjectkey = description_row.apply(lambda x: 'promoted_subjectkey' in str(x)).any()
    cols_contain_collection_title = 'collection_title' in recombined_df.columns
    cols_contain_promoted_subjectkey = 'promoted_subjectkey' in recombined_df.columns

    if not desc_contains_collection_title:
        print("Collection_title is missing from the column descriptions.")
    if not desc_contains_promoted_subjectkey:
        print("Promoted_subjectkey is missing from the column descriptions.")
    if not cols_contain_collection_title:
        print("Collection_title is missing from the column names.")
    if not cols_contain_promoted_subjectkey:
        print("Promoted_subjectkey is missing from the column names.")
    if desc_contains_collection_title and desc_contains_promoted_subjectkey and cols_contain_collection_title and cols_contain_promoted_subjectkey:
        return recombined_df

def process_all_files(root_folder, file_list, text_columns_start_row=0):
    """
    Processes all files in the list and returns the processed DataFrames.
    
    Parameters:
    - root_folder: Root folder containing all text files.
    - file_list: List of file names to process.
    - text_columns_start_row: Row index from where to start processing text columns (default is 1).
    
    Returns:
    - result_dict: Dictionary with file names as keys and processed DataFrames as values.
    """
    result_dict = {}
    
    for file_name in file_list:
        file_path = os.path.join(root_folder, file_name)
        print(f"Processing {file_name}...")
        
        # Process the file
        processed_df = process_file(file_path, text_columns_start_row)
        
        # Store the result in the dictionary
        if isinstance(processed_df, pd.DataFrame):
            result_dict[file_name] = processed_df
    
    return result_dict



In [105]:
#Get a list of all text files from the root folder

def get_text_files(directory):
    text_files = [f for f in os.listdir(directory) if f.endswith('.txt') and os.path.isfile(os.path.join(directory, f))]
    return text_files

root_folder = r"/Users/sennakim/Desktop/catie/CATIE_side_effects/catie_text"

file_list = get_text_files(root_folder)
print(file_list)

['lab01.txt', 'hair01.txt', 'fint01.txt', 'sf1201.txt', 'ndar_subject01.txt', 'clgry01.txt', 'panss01.txt', 'sae01.txt', 'itaq01.txt', 'viol01.txt', 'cata01.txt', 'meddispn01.txt', 'dgsposys01.txt', 'ecg01.txt', 'surfq01.txt', 'endstudy01.txt', 'package_info.txt', 'ndar_aggregate.txt', 'aesposys01.txt', 'macvlnce01.txt', 'timeto01.txt', 'med01.txt', 'maccomp01.txt', 'aims01.txt', 'dai01.txt', 'dosecomp01.txt', 'vitals01.txt', 'scid_ph01.txt', 'surf01.txt', 'neurobatt01.txt', 'demo01.txt', 'screen01.txt', 'keyvars01.txt', 'endphase01.txt', 'qol01.txt', 'cgis01.txt']


In [106]:
# Process all files and store the results in a dictionary
processed_data = process_all_files(root_folder, file_list)
# processed_data['med01.txt']

Processing lab01.txt...
Processing hair01.txt...
Processing fint01.txt...
Processing sf1201.txt...
Processing ndar_subject01.txt...
Processing clgry01.txt...
Processing panss01.txt...
Processing sae01.txt...
Processing itaq01.txt...
Processing viol01.txt...
Processing cata01.txt...
Processing meddispn01.txt...
Processing dgsposys01.txt...
Processing ecg01.txt...
Processing surfq01.txt...
Processing endstudy01.txt...
Processing package_info.txt...
Collection_title is missing from the column descriptions.
Promoted_subjectkey is missing from the column descriptions.
Collection_title is missing from the column names.
Promoted_subjectkey is missing from the column names.
Processing ndar_aggregate.txt...
Collection_title is missing from the column descriptions.
Promoted_subjectkey is missing from the column descriptions.
Collection_title is missing from the column names.
Promoted_subjectkey is missing from the column names.
Processing aesposys01.txt...
Processing macvlnce01.txt...
Processing

In [107]:
#Merge all dfs which have promoted_subjectkey and a collection_title
#This is going to merge on all shared columns, doing outer to make sure all rows are kept. NaNs will go in where info does not exist for that row.

# Initial DataFrame (use the first DataFrame in the dictionary)
merged_df = list(processed_data.values())[0]

# Merge remaining DataFrames
for key in list(processed_data.keys())[1:]:
    merged_df = pd.merge(merged_df, processed_data[key], how='outer')

In [108]:
#other idea of what the column of medications for phase 1 is
#merged_df.loc['med2g']
merged_df['promoted_subjectkey'].value_counts()

promoted_subjectkey
NDAR_INVYR649KT1       2181
NDAR_INVUJ397VYA       1786
NDAR_INVUK248FD1       1738
NDAR_INVUN857PW0       1586
NDAR_INVBG113KYW       1519
NDAR_INVKC621FX1       1439
NDAR_INVBM981LV1       1423
NDAR_INVTY650WF3       1396
NDAR_INVZY888ZC7       1396
NDAR_INVZB079KUC       1383
NDAR_INVBD974GTH       1368
NDAR_INVYL522DTK       1361
NDAR_INVRM608VCY       1344
NDAR_INVKF983CB0       1334
NDAR_INVML850AYA       1327
NDAR_INVNU972TNA       1322
NDAR_INVTT723NPW       1304
NDAR_INVTG828ZF9       1300
NDAR_INVJR448AWR       1300
NDAR_INVLK707KDG       1295
NDAR_INVRN808RGP       1287
NDAR_INVUB310RU6       1263
NDAR_INVWU491FFH       1258
NDAR_INVXH328AN2       1240
NDAR_INVWM052HDK       1218
NDAR_INVYA067HP5       1213
NDAR_INVKD155AP4       1212
NDAR_INVXX208GER       1199
NDAR_INVGM160JV7       1187
NDAR_INVZJ340TPB       1172
NDAR_INVXU990CJK       1133
NDAR_INVJM997GKH       1131
NDAR_INVYJ087RDW       1125
NDAR_INVKW099DZA       1122
NDAR_INVMD062XMR       1116


In [109]:
# for each promoted subject key, get rows with non-NAN values of: treat_1, dcr_tae1, dcr_eff1
# with that promoted subject key as 'key', find the values for 'cocaine', 'opiates', 'pcp', 'meth', 'thc'.
# do logistic regression. 

In [110]:
# Step 1: Filter for non-NA 'treat_1' and get relevant 'promoted_subjectkey'
#filtered_df = merged_df[merged_df['treat_1'].notna()]
#promoted_keys = filtered_df['promoted_subjectkey'].unique()

In [111]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

# Display the first two rows
#merged_df.head()

In [ ]:
# Step 2: Filter for the relevant 'promoted_subjectkey' and required columns
selected_df = merged_df[merged_df['promoted_subjectkey'].isin(promoted_keys)]

# Step 2b: Aggregate data by 'promoted_subjectkey', retaining the first value for all variables
agg_df = selected_df.groupby('promoted_subjectkey').agg('first').dropna(subset=['dcr_tae1'])  # Ensure 'dcr_tae1' has values


In [ ]:
# Adding 2nd row of variable descriptions.
# First create a single row of NaNs to be concatenated to merged_df
#null_row = pd.DataFrame([np.nan] * len(agg_df.columns)).T
#null_row.columns = agg_df.columns

In [ ]:
#concat merged_df with the NaN's as 2nd row
#agg_df= pd.concat([agg_df.iloc[:1], null_row, agg_df.iloc[1:]]).reset_index(drop=True)

In [ ]:
# Create a row of NaN values with the same columns as 'agg_df'
null_row = pd.DataFrame([[np.nan] * agg_df.shape[1]], columns=agg_df.columns)

# Concatenate the null_row as the first row in the aggregated dataframe
agg_df = pd.concat([null_row, agg_df], ignore_index=True)

# Check the updated dataframe
agg_df.head()

In [ ]:
# Mapping for the variable descriptions
variable_map = {
    'eps': 'Extrapyramidal symptoms',
    'CGIS': 'clinical global impression scale; severity of psychopathology on a scale of 1 to 7',
    'ECG' : 'electrocardiogram',
    'QT': "ECG reading; beginning of Q wave to end of T wave",
    'SF_12': 'Medical Outcome Study Short Form-12; 12 item questionnaire measures general health status and functioning',
    'BRS':'Barnses Akathisia Scale; movement disorder severity',
    'AE' : 'Adverse events',
    'Calgary' : ' Calgary depression scare for Schizophrenia',
    'DAI' : 'drug attitude inventory',
    'ITAQ' : 'insight into treatment attitude questionnaire',
    'WRAT-3' : 'wide range achievement test-3, reading subset',
    'COWAT' : 'controlled oral word association test'
    # Add more mappings
}

def variable_description(df, variable_map):
    # Work only on the first two rows
    for col in df.columns:
        first_row_value = df.at[0, col]
        print(f"Checking column: {col}, first row value: {first_row_value}")  # Debugging line
        
        # Check if any of the words in variable_map are present in the first row value
        for key, value in variable_map.items():
            if pd.notna(first_row_value) and key in str(first_row_value):  
                print(f"Found match for {key} in column {col}")  # Debugging line
                # Check if the second row is NaN and populate it with the mapped description 
                if pd.isna(df.at[1, col]):
                    print(f"Updating second row value in column {col} with: {value}")  # Debugging line
                    df.at[1, col] = value

    return df  # Return the modified dataframe

# Example usage:
agg_df = variable_description(agg_df, variable_map)


In [ ]:
# Apply function
variable_description(agg_df, variable_map)

In [ ]:
merged_df.columns.get_loc('epsmean')

In [ ]:

# test
print(agg_df.iloc[:3, 700:706])